# Описание исследования

Проводим работу с табличными данными, в которых представлена информация о стартапах, которые функционировали в период с 1980 по 2018 годы. Нужно предсказать, какие из них закроются, а какие нет. Соревнование проводится на популярной платформе Kaggle, что позволит не только применять на практике свои знания в области анализа данных и машинного обучения, но и освоить работу с этой платформой. 

# Цель исследования

- Разработать модель машинного обучения для предсказания продолжения деятельности стартапа. 
- Провести полноценный разведочный анализ и сформировать рекомендации будущим создателям стартапов (какие факторы влияют на успешность стартапа).

# Этапы исследования

- загрузка и ознакомление с данными,
- предварительная обработка,
- полноценный разведочный анализ,
- разработка новых синтетических признаков,
- проверка на мультиколлинеарность,
- отбор финального набора обучающих признаков,
- выбор и обучение моделей,
- итоговая оценка качества предсказания лучшей модели,
- анализ важности ее признаков,
- подготовка отчета по исследованию.

# Описание данных

Датасет состоит из двух файлов: 
- тренировочный набор (около 53к записей) и тестовый набор (около 13к записей). 
- Тренировочный набор содержит целевой признак status, указывающий на то, закрылся стартап или продолжает действовать. 
Временной период - '1970-01-01' по '2018-01-01'. 
Дата формирования выгрузки - '2018-01-01'

- `kaggle_startups_train_27042024.csv` - информация (53 000) стартапах, которые будут использоваться в качестве обучающих данных.
- `kaggle_startups_test_27042024.csv` - информация (13 000) стартапах, которые будут использоваться в качестве тестовых данных. Ваша задача - предсказать значение 'status' для каждого стартапа из этого датасета.
- `kaggle_startups_sample_submit_27042024.csv` - файл с примером предсказаний в правильном формате.
name - идентификатор (название) стартапа в тестовом наборе.
status - целевой признак. Для каждого стартапа предскажите категориальное значение соответствующее прогнозу ['operating', 'closed']. 

**Описание полей данных**
- `name` - Название стартапа
- `category_list` - Список категорий, к которым относится стартап
- `funding_total_usd` - Общая сумма финансирования в USD
- `status` - Статус стартапа (закрыт или действующий)
- `country_code` - Код страны
- `state_code` - Код штата
- `region` - Регион
- `city` - Город
- `funding_rounds` - Количество раундов финансирования
- `founded_at` - Дата основания
- `first_funding_at` - Дата первого раунда финансирования
- `last_funding_at` - Дата последнего раунда финансирования
- `closed_at` - Дата закрытия стартапа (если применимо)
- `lifetime` - Время существования стартапа в днях

In [ ]:
! pip install phik -q
! pip install shap -q
! pip install -U scikit-learn -q
! pip install seaborn --upgrade -q
! pip install missingno -q

In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

In [ ]:
import os
import matplotlib.pyplot as plt
import missingno as msno
import pandas as pd
import numpy as np
import seaborn as sns
import shap
from sklearn.dummy import DummyClassifier


from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    MinMaxScaler,
)
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
import phik  # noqa: F401
from sklearn.experimental import enable_hist_gradient_boosting  # noqa: F401

from catboost import CatBoostClassifier

In [ ]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", "{:.3f}".format)

In [ ]:
def preprocessing(df):
    print("Изучение данных датафрейма")
    print("Вывод первых 5 строк \n")
    display(df.head(5))
    print("-" * 100)
    print(f"Размерность датафрейма, составляет - {(df.shape)}")
    print("-" * 100)
    print("Общая информация:\n")
    display(df.info())
    print("-" * 100)
    print("Описательная статистика:\n")
    display(df.describe())
    print("-" * 100)
    df.isna().sum()
    print("Проверка на наличие явных дубликатов:\n")
    display(df.duplicated().sum())
    print("-" * 100)
    print("Вывод количества уникальных значений")
    display(pd.DataFrame(df.apply(lambda x: x.nunique())))

In [ ]:
pth1 = "./datasets/kaggle_startups_train_27042024.csv"
pth2 = "./datasets/kaggle_startups_test_27042024.csv"

if os.path.exists(pth1):
    # Тренировочная выборка:
    df_ks_train = pd.read_csv(
        pth1,
        parse_dates=["founded_at", "first_funding_at", "last_funding_at", "closed_at"],
    )
    if os.path.exists(pth2):
        # Тестовая выборка:
        df_ks_test = pd.read_csv(
            pth2, parse_dates=["founded_at", "first_funding_at", "last_funding_at"]
        )
    else:
        print("Something is wrong")

## Загрузка и ознакомление с данными

Выведем первоначальную информацию о датафрейме с тренировочными данными

In [ ]:
preprocessing(df_ks_train)

Посмотрим количество пропусков в тренировочных данных

In [ ]:
nan_counts = df_ks_train.isna().sum()
print(nan_counts)

In [ ]:
# Создание объекта осей
fig, ax = plt.subplots(figsize=(18, 4))
# Генерация графика с использованием объекта осей
msno.matrix(df_ks_train, ax=ax, sparkline=False)
# Добавление названия графика
ax.set_title("Матрица пропущенных значений")
# Отображение графика
plt.show()

**Что можно сказать из первоначальных данных:**
- `name` - имеет 1 пропуск, удалим так как он не испорит общую картину
- `category_list`   - список категорий -  имеет большое количество пропусков, подумаем что с ними можно сделать - 2465, возможно просто внесем категорию Unknown, так же иммет огромное количество категорий, но множество из них разделено знаком |, предлагаю удалить данные после данного символа и оставить название категории до вышеуказанного символа. Позволит уменьшить количество уникальных категорий.
- `funding_total_usd`   - Общая сумма финансирования в USD - тоже множество пропусков  - 10069, предлагаю заполнить медианными значениями по группе категория стартапа. Не имеющие данных либо категорию неизвестно - заполнить глобальным медианным значением.
- `status`   - статус. Не имеет проблем в данных. Это наш целевой признак.
- `country_code`     - код старны - имеет много пропусков 5501, чем заполнить пока не ясно, оставим, так же имеет не так много категорий, переведем в категориальный тип. Предлагаю заполнить категорией Unknown, а те коды группа которых составляет меньше 10 - сделать как - Other.
- `state_code`      state_code - Код штата    имеет много пропусков  6762, чем заполнить пока не ясно, оставим, так же имеет не большое количество категорий, переведем в категориальный тип. Думаю данная колонка не нужна нам для обучения модели, так как согласно матрице пропусков, это все географические названия и пропуски связаны между собой. Для обучения будем использовать код страны.
- `region`        - региона, много пропусков -  6358, и много категорий. Думаю дальше данные эти не понадобятся для обучения модели, оставим пропуски. Думаю данная колонка не нужна нам для обучения модели, так как согласно матрице пропусков, это все географические названия и пропуски связаны между собой. Для обучения будем использовать код страны
- `city`           - город, много пропусков -        6358, соответствует данным по региону. Вообще все пропуски связанные с географическим положением не заполнены в одинх и теж же строках, это можно наглядно увидеть по матрице пропусков. Возможно кто то заполнял только код страны, а остальные данные были не обязательны для заполнения, либо данные о них просто отсутствовали. Думаю данная колонка не нужна нам для обучения модели, так как согласно матрице пропусков, это все географические названия и пропуски связаны между собой. Для обучения будем использовать код страны 
- `funding_rounds`  -   Количество раундов финансирования    - Не имеет проблем в данных. Только переведем в int, так как даные числовые.
- `founded_at`       -    Дата основания    - Не имеет проблем в данных.
- `first_funding_at`  -     Дата первого раунда финансирования - Не имеет проблем в данных.
- `last_funding_at`    -      Дата последнего раунда финансирования - Не имеет проблем в данных.
- `closed_at`        -    Дата закрытия стартапа (если применимо) - имеет большое количество пропусков - 47599, что может говорит о том, что большинство из представленных в данных стартапов являются действующими. Данная колонка будет мешать предсказанию, так как нам надо именно предсказать будет ли закрыт стартап или нет, и оставив ее мы получим утечку целевого признака, поэтому заполним пропуски датой выгрузки данных и создадим новую синтетическую информацию, а именно `lifetime` - Время существования стартапа в днях, для этого посчитаем количество дней прошедших между датой основания и датой закрытия

## Предварительная обработка даных

### Приведение типов данных

In [ ]:
df_ks_train = df_ks_train.astype(
    {"funding_rounds": np.int32, "funding_total_usd": np.float32}
)
df_ks_train[df_ks_train.select_dtypes(["object"]).columns] = df_ks_train.select_dtypes(
    ["object"]
).apply(lambda x: x.astype("category"))
df_ks_train.info()

### Работа с пропусками

#### **Поработаем с пропусками в колонке name**

In [ ]:
missing_name_row = df_ks_train[df_ks_train["name"].isna()]
missing_name_row

Удалим ее и сбросим индексы

In [ ]:
df_ks_train = df_ks_train.dropna(subset=["name"]).reset_index(drop=True)

In [ ]:
# Проверим на пропуски
df_ks_train.isnull().sum()

#### **Поработаем с колонкой category_list**

В данной колонке очень много уникальных значений, но почти все они имеют разделитиель в качестве знака | , укрупним группы, оставив только первое наименование категории как самое крупное

In [ ]:
# Используем .loc[] для обновления столбца 'category_list'
df_ks_train.loc[:, "category_list"] = df_ks_train["category_list"].str.split("|").str[0]
# Подсчитываем количество уникальных категорий
unique_categories = df_ks_train["category_list"].nunique()
# Выводим количество уникальных категорий
unique_categories

У нас существенно уменьшилось количество категорий к которым относятся стартапы с 22108 до 707

Заменим пропуски в данной категории заглушкой "Unknown"

In [ ]:
df_ks_train["category_list"] = df_ks_train["category_list"].fillna("Unknown")

Посчитаем количество уникальных значений в каждой категории

In [ ]:
category_counts = df_ks_train["category_list"].value_counts()
category_counts

Видим что некоторые категории содержат по 1 включению, это помешает обучать модель, да и в принципе такие данные не информативны, заменим такие категории на Other. Порогом будет менее 5 включений

In [ ]:
df_ks_train["category_list"] = df_ks_train["category_list"].apply(
    lambda x: "Other" if category_counts[x] < 5 else x
)

In [ ]:
df_ks_train["category_list"].nunique()

Смогли уменьшить количество категорий до 478, считаю это вполне успешным.

**Построим график ТОП 10 категорий стартапов**

In [ ]:
# Создаем фильтрованный датафрейм без категории 'Unknown'
filtered_df = df_ks_train[df_ks_train["category_list"] != "Unknown"]

# Подсчитываем количество проектов по категориям, исключая 'Unknown'
category_counts = filtered_df["category_list"].value_counts().head(10)

# Строим график топ-10 категорий по количеству проектов
category_counts.plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Топ-10 категорий по количеству проектов")
plt.xlabel("Категории")
plt.ylabel("Количество проектов")
plt.gca().spines[["top", "right"]].set_visible(False)
plt.show()

Смогли снизить количество категорий, а так же визуализировали наиболее часто встречающиеся стартапы.

#### **Поработаем с колонкой status**

In [ ]:
# Проверим на пропуски
df_ks_train.status.isnull().sum()

In [ ]:
category_counts = df_ks_train.status.value_counts()
plt.bar(
    category_counts.index, category_counts.values, color="skyblue", edgecolor="black"
)

# Добавляем название диаграммы и осей
plt.title("Текущий статус стартапа")
plt.xlabel("Статус")
plt.ylabel("Количество")

# Отображаем график
plt.gca().spines[["top", "right"]].set_visible(False)
plt.show()

Пропущенных значений нет, большинство стартапов являются действующими

#### **Поработаем с колонкой country_code**

In [ ]:
unique_categories_country_code = df_ks_train["country_code"].nunique()
unique_categories_country_code

In [ ]:
new_var = df_ks_train["country_code"].unique()
new_var

Заменим пропуски на заглушку 'Unknown'

In [ ]:
# Добавляем 'Unknown' в категории
df_ks_train["country_code"] = df_ks_train["country_code"].cat.add_categories(
    ["Unknown"]
)

# Заменяем NaN на 'Unknown'
df_ks_train["country_code"] = df_ks_train["country_code"].fillna("Unknown")

Посчитаем количество уникальных значений в каждой категории

In [ ]:
counts = df_ks_train["country_code"].value_counts()
counts

Снова видим много стран имеющих только по 1 включению, используем тот же подход и заменим на Other

In [ ]:
df_ks_train["country_code"] = df_ks_train["country_code"].apply(
    lambda x: "Other" if counts[x] < 2 else x
)

In [ ]:
df_ks_train["country_code"].nunique()

Построим график ТОП-10 Кодов стран, без учета "Unknown"

In [ ]:
# Фильтруем DataFrame, чтобы исключить строки с 'Unknown'
filtered_df = df_ks_train[df_ks_train["country_code"] != "Unknown"]
# Подсчитываем количество проектов по странам, исключая 'Unknown'
country_counts = filtered_df["country_code"].value_counts().head(10)
# Строим график топ-10 стран по количеству проектов
country_counts.plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("топ-10 стран по количеству проектов")
plt.xlabel("Страны")
plt.ylabel("Количество проектов")
plt.gca().spines[["top", "right"]].set_visible(False)
plt.show()

Снизили количество категорий с 134 до 114 (возможно стоит взять порог побольше, но пока посмотрим так)

#### **Поработаем с колонкой state_code**

In [ ]:
unique_categories_state_code = df_ks_train["state_code"].nunique()
unique_categories_state_code

In [ ]:
df_ks_train["state_code"].unique()

Имеем множество уникальных значений, где есть и цифры и буквы и их комбинации. Попробуем уменьшить их количество объединив наиболее мелкие категории содержащие до 10 значений в одну категорию Other

In [ ]:
counts = df_ks_train["state_code"].value_counts()
threshold = 10  # Порог
df_ks_train["state_code"] = df_ks_train["state_code"].apply(
    lambda x: "Other" if counts[x] < threshold else x
)

In [ ]:
df_ks_train["state_code"].nunique()

Получили существенное уменьшение категорий, заполним пропуски значением Unknown. Полагаю что данная колонка не понадобится  нам для обучения, поэтому можно просто поставить заглушку чтоб сохранить данные в других столбцах

In [ ]:
# Фильтруем DataFrame, чтобы исключить строки с 'Unknown' и 'Other'
filtered_df_2 = df_ks_train[
    (df_ks_train["state_code"] != "Other") & (df_ks_train["state_code"] != "Unknown")
]
# Подсчитываем количество проектов по странам, исключая 'Unknown' и 'Other'
state_counts = filtered_df_2["state_code"].value_counts().head(10)
# Строим график топ-10 по количеству проектов
state_counts.plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("топ-10 кодов штата по количеству проектов")
plt.xlabel("Коды штатов")
plt.ylabel("Количество проектов")
plt.gca().spines[["top", "right"]].set_visible(False)
plt.show()

Открытые источники говорят что код штата СА - Калифорния, NY - Нью-Йорк, а МА - Массачусетс
В принципе ТОП 3 кодов штатов, подтверждает предыдущий график, что больше всего стартапов в США, и уже внутри США самым оживленным по стартапам является Калифорния

#### **Поработаем с колонкой funding_total_usd**

In [ ]:
df_ks_train["funding_total_usd"].describe()

Видим явный выброс, как значение 30 079 502 336, при том что медиана у нас всего 2 000 000 долларов, посмотрим на это значение и предлагаю удалить его. Так же предлагаю посмотреть значения которые выпадают за 99 квантиль и тоже удалить их, так как это будет не более 1% от выборки, а данные существенно улучшатся

In [ ]:
# Вычисление 99-го квантиля для столбца 'funding_total_usd'
quantile_99 = df_ks_train["funding_total_usd"].quantile(0.99)
print(
    f' значения выше чем {quantile_99} выпадают за 99 квантиль их количество составляет {len(df_ks_train.loc[df_ks_train["funding_total_usd"] > quantile_99])}'
)
# Создание боксплота с ограничением данных 99-м квантилем
fig, ax = plt.subplots(figsize=(10, 4))
sns.boxplot(
    x="funding_total_usd",
    data=df_ks_train[df_ks_train["funding_total_usd"] <= quantile_99],
    orient="h",
    ax=ax,
)
ax.ticklabel_format(style="plain", axis="x")
ax.set_title("Боксплот")
ax.set_xlabel("Общая сумма финансирования в USD (ограничено 99 квантилем)")
plt.gca().spines[["top", "right"]].set_visible(False)
plt.show()

In [ ]:
df_ks_train = df_ks_train[
    (df_ks_train["funding_total_usd"] <= quantile_99)
    | (df_ks_train["funding_total_usd"].isna())
]

После удаления значений выпадающих за 99 квантиль заполним оставщиеся значения по следующему принципу.
Заполним пропущенные данные медианой по группе категорий.  Так как мы не знаем в какие категории могли попасть в 'Unknown', то заполним ее не медианой по группе 'Unknown', а глобальной медианой, для этого:
Создадим функцию fill_with_median которая проверяет, равна ли категория - 'Unknown', и если это так, то она заполняет пропущенные значения в funding_total_usd глобальной медианой. Это гарантирует, что для категории 'Unknown' всегда будет использоваться глобальная медиана, а не медиана по группе 'Unknown'. Для остальных значений сначала проверяет есть ли в группе хоть 1 значение, если есть заполняет медианой по группе, в противном случае глобальной медианой.

In [ ]:
# Вычисление глобальной медианы для столбца 'funding_total_usd'
global_median = df_ks_train["funding_total_usd"].median()


def fill_with_median(group, category):
    # Если категория группы - 'Unknown', заполнение NaN значений глобальной медианой
    if category == "Unknown":
        return group.fillna(global_median)
    # Проверка наличия хотя бы одного не-NaN значения в группе
    elif group.notna().any():
        # Заполнение NaN значений медианой группы
        return group.fillna(group.median())
    else:
        # Заполнение NaN значений глобальной медианой
        return group.fillna(global_median)


# Применение функции fill_with_median к каждой группе в столбце 'category_list'
df_ks_train["funding_total_usd"] = df_ks_train.groupby("category_list")[
    "funding_total_usd"
].transform(lambda x: fill_with_median(x, x.name))

In [ ]:
df_ks_train.isna().sum()

## Разработка новых синтетических признаков

**Создание признака lifetime - Время существования стартапа в днях**

Заполним closed_at датой выгрузки данных, для последующего создания столбца lifetime - Время существования стартапа в днях

In [ ]:
df_ks_train["closed_at"] = df_ks_train["closed_at"].fillna(datetime(2018, 1, 1))

In [ ]:
df_ks_train["lifetime"] = (df_ks_train["closed_at"] - df_ks_train["founded_at"]).dt.days
df_ks_train.head(5)

closed_at	нам больше не нужен, удалим его

In [ ]:
df_ks_train = df_ks_train.drop("closed_at", axis=1)

Проверим, что все удалилось, а нужные нам столбцы остались

In [ ]:
df_ks_train.head(5)

**Создадим новый признак который бы показывал какое количество дней прошло между созданием стартапа и получения первого финансирования - first money**

In [ ]:
df_ks_train["first_money"] = (
    df_ks_train["first_funding_at"] - df_ks_train["founded_at"]
).dt.days
df_ks_train.head(5)

**Создадим новый признак который бы показывал как часто финансироался стартам, для этого разницу между 1 и последним раундом финансирования разделим на количество раундов финансирования - frequency_financing**

In [ ]:
df_ks_train["frequency_financing"] = (
    (df_ks_train["last_funding_at"] - df_ks_train["first_funding_at"]).dt.days
) / df_ks_train["funding_rounds"]
df_ks_train.head(5)

In [ ]:
df_ks_train.info()

Итак у нас дополнительно создано 3 синтетических признака, которые должны помочь в обучении модели

**Посмотрим визуализации изменения количества раундов финансирования компаний в зависимости от даты их основания**

In [ ]:
def plot_series(series):
    xs = series["founded_at"]
    ys = series["funding_rounds"]
    plt.plot(xs, ys)


fig, ax = plt.subplots(figsize=(10, 5.2), layout="constrained")
df_sorted = df_ks_train.sort_values("founded_at", ascending=True)
plot_series(df_sorted)
sns.despine(fig=fig, ax=ax)
plt.xlabel("Дата основания компании")
plt.ylabel("Раунд финансирования")
plt.title("зависимость количества раундов финансирования от даты основания компаний")
plt.show()

Данный график может сказать, что компании в основном имеют до 7 раундов финансирования, а большая часть финансирования началась для компаний созданных после 2000 года

In [ ]:
df_ks_train["lifetime"].plot(
    kind="hist", bins=50, title="Гистограмма времени жизни стартапа"
)
plt.xlabel("Время жизни стартапа")
plt.ylabel("Частота")
plt.gca().spines[["top", "right"]].set_visible(False)
plt.show()

Исходя из данной гистограммы видим что время жизни стартапа в основном составляет около 2000 дней, что достаточно много, так же наблюдаются стартапы долгожители более 10 000 дней

## **Проведем корреляционный анализ данных, проверив данные на мультиколлинеарность**

Используем библиотеку FINK, но исключим данные связанные датами, такими как дата основания, первый и последний раунд финансирования. Основна временная позиция - это время жизни стартапа, думаю это будет основа нашего будущего предсказания.   
*P.S. Если использовать все данные в таблице, расчет проводиться очень долго, поэтому оставлю для примера код страны и штата чтоб обозначить их сильную связь.*

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(
    df_ks_train.drop(
        [
            "name",
            "state_code",
            "country_code",
            "city",
            "founded_at",
            "first_funding_at",
            "last_funding_at",
        ],
        axis=1,
    ).phik_matrix(verbose=False),
    annot=True,
    vmin=-1,
    vmax=1,
    cmap="coolwarm",
    linewidths=1,
    linecolor="black",
)
plt.title("Коэффициент корреляции $\phi_K$")
plt.show()

Наибольшую взаимосвязь видим между целевым признаком количеством дней жизни стартапа - 0,54.

Так же имеется достаточно сильная взаимосвязь между категорией и регионом - 0,74, временем жизни стартапа и количеством дней до первой даты финансирования - 0,88

При мультиколлинеральноси имеются взаимосвязи т 0.9 до 0.95 по модулю. 

## Отбор финального набора обучающих признаков

- status  - целевой признак
- category_list - категория
- funding_total_usd - объем финансирования
- country_code - код страны
- funding_rounds - раунд финансирования
- lifetime - время жизни стартапа
- first_money - получение первого финансирования
- frequency_financing  - частота получения финансирования

In [ ]:
feature = [
    "name",
    "status",
    "category_list",
    "funding_total_usd",
    "country_code",
    "funding_rounds",
    "lifetime",
    "first_money",
    "frequency_financing",
]

Создадим датафрейм содержащий только данные столбцы для обучения

In [ ]:
df_train = df_ks_train[feature]
df_train.info()

Убедимся что у нас не осталось пропусков в значениях необходимых для обучения

In [ ]:
df_train.isna().sum()

## Проведение аналогичной предобработки данных на тестовых (приведение к одному виду и размерности)

In [ ]:
preprocessing(df_ks_test)

In [ ]:
nan_counts_test = df_ks_test.isna().sum()
nan_counts_test

Имеются пропуски в используемых для обучения аналогичных колонках 
- category_list         591
- funding_total_usd    2578
- country_code         1382  

Так как строки удалять нельзя (заполним пропуски)

- category_list       - первым значением по категориям, остальные категорией Unknown
- funding_total_usd    - медианным значением по категории
- country_code         - категорией Unknown

In [ ]:
df_ks_test.loc[:, "category_list"] = df_ks_test["category_list"].str.split("|").str[0]
df_ks_test["category_list"] = df_ks_test["category_list"].fillna("Unknown")

In [ ]:
# Применение функции fill_with_median к каждой группе в столбце 'category_list'
df_ks_test["funding_total_usd"] = df_ks_test.groupby("category_list")[
    "funding_total_usd"
].transform(lambda x: fill_with_median(x, x.name))

In [ ]:
# Заменяем NaN на 'Unknown'
df_ks_test["country_code"] = df_ks_test["country_code"].fillna("Unknown")

In [ ]:
df_ks_test.isna().sum()

#### **Создаем новые синтетические признаки из имеющихся, аналогичные тренировочному датасету**

In [ ]:
df_ks_test["first_money"] = (
    df_ks_test["first_funding_at"] - df_ks_test["founded_at"]
).dt.days
df_ks_test["frequency_financing"] = (
    (df_ks_test["last_funding_at"] - df_ks_test["first_funding_at"]).dt.days
) / df_ks_test["funding_rounds"]
df_ks_test.isna().sum()

Для поддержания размерности оставить в тестовом датфрейме только столбцы аналогичные тренировочному (за исключением статус - его мы предсказываем)

In [ ]:
feature_test = [
    "name",
    "category_list",
    "funding_total_usd",
    "country_code",
    "funding_rounds",
    "lifetime",
    "first_money",
    "frequency_financing",
]
df_test = df_ks_test[feature_test]
df_test.info()

Проверим размерость тествого и тренировочно, разница должна быть в один столбец (статус еще не создан в тестовом)

In [ ]:
display(df_train.shape)
df_test.shape

## Выбор и обучение моделей

Так как данные несбалансированы, используем stratify=y, обеспечивая, что каждая выборка будет иметь представительное распределение классов, позволяя модели обучаться и тестироваться на данных, которые лучше отражают реальное распределение классов в исходном наборе данных. Это помогает улучшить надежность и обобщаемость модели машинного обучения.

In [ ]:
# Фичи для обучения
X = df_train.drop(["name", "status"], axis=1)
y = df_train["status"]  # Метки классов

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
categorical_features = ["category_list", "country_code"]
numeric_features = df_train.select_dtypes(include=[np.number]).columns.tolist()

### Используем Pipeline для обучения модели

ПЕРЕБОР МОДЕЛЕЙ

In [ ]:
# Создадим препроцессор - предварительный обработчик данных с помощью ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        (
            "cat",
            OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False),
            categorical_features,
        ),
    ],
    remainder="passthrough",
)

In [ ]:
# Создадим итоговый пайплайн куда будем передавать предобработчик и модели с разными параметрами
pipe_final = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("models", DecisionTreeClassifier(random_state=42)),
    ]
)

In [ ]:
param_distributions = [
    # словарь для модели HistGradientBoostingClassifier
    {
        "models": [HistGradientBoostingClassifier()],
        "models__max_iter": range(100, 150),
        "models__learning_rate": [0.001, 0.01, 0.05, 0.1],
        "models__max_depth": range(3, 5),
        "preprocessor__num": [StandardScaler(), MinMaxScaler(), "passthrough"],
    },
    # словарь для модели DecisionTreeClassifier()
    {
        "models": [DecisionTreeClassifier(random_state=42)],
        "models__max_depth": range(2, 6),
        "models__max_features": range(2, 6),
        "preprocessor__num": [StandardScaler(), MinMaxScaler(), "passthrough"],
    },
    # словарь для модели LogisticRegression()
    {
        "models": [
            LogisticRegression(random_state=42, solver="liblinear", penalty="l1")
        ],
        "models__C": range(1, 10),
        "preprocessor__num": [StandardScaler(), MinMaxScaler(), "passthrough"],
    },
    # словарь для модели GradientBoostingClassifier
    {
        "models": [GradientBoostingClassifier(random_state=42)],
        "models__n_estimators": (50, 150),
        "models__learning_rate": (0.1, 1),
        "models__max_depth": (3, 6),
        "models__min_samples_split": (2, 4),
        "models__min_samples_leaf": (1, 3),
        "preprocessor__num": [StandardScaler(), MinMaxScaler(), "passthrough"],
    },
    # словарь для модели CatBoostClassifier
    {
        "models": [CatBoostClassifier(random_state=42, silent=True)],
        "models__depth": range(2, 6),
        "models__learning_rate": [0.03, 0.1, 0.3],
        "models__iterations": [100, 200, 300],
        "preprocessor__num": [StandardScaler(), MinMaxScaler(), "passthrough"],
    },
]

In [ ]:
randomized_search = RandomizedSearchCV(
    pipe_final,
    param_distributions=param_distributions,
    scoring="f1_weighted",
    random_state=42,
    n_jobs=-1,
)

In [ ]:
%%capture --no-stdout
randomized_search.fit(X_train, y_train)

In [ ]:
# Используем лучшую найденную модель для предсказания
preds = randomized_search.predict(X_test)
f1 = f1_score(y_test, preds, average="weighted")
print(f"Метрика F1 на тестовой выборке: {f1}")

In [ ]:
print("Лучшая модель и её параметры:")
display(randomized_search.best_estimator_)
print("Лучшие параметры модели:", randomized_search.best_params_)
print("Метрика лучшей модели на тренировочной выборке:", randomized_search.best_score_)

In [ ]:
pd.DataFrame(randomized_search.cv_results_)[
    ["params", "std_test_score", "rank_test_score", "param_models", "mean_test_score"]
].sort_values("rank_test_score").head(3)

In [ ]:
best_model = (
    randomized_search.best_estimator_
)  # вот это лучшая модель (preprocessor + estimator), уже обученая.
regressor = best_model.named_steps["models"]  # Это отдельно модель
preprocessor = best_model.named_steps["preprocessor"]  # Это препроцессор

### Проверим адекватность модели с помощью Dummy модели

In [ ]:
# Обучение DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
dummy_pred = dummy_clf.predict(X_test)

In [ ]:
print("DummyClassifier F1 Score:", f1_score(y_test, dummy_pred, average="weighted"))
print("Лучшая модель дает -  F1 Score:", f1_score(y_test, preds, average="weighted"))

Результаты показывают, что наша модель имеет F1-меру 0.988, что значительно выше, чем F1-мера DummyClassifier, равная 0.861. Это указывает на то, что CatBoostClassifier работает лучше, чем простой классификатор, который делает предсказания, основываясь только на самом распространенном классе.

F1-мера является гармоническим средним между точностью и полнотой, и она особенно полезна, когда распределение классов несбалансировано.     
Высокое значение F1-меры для CatBoostClassifier говорит о том, что модель хорошо справляется с балансом между точностью и полнотой, что делает ее адекватной для нашей задачи классификации.

### Определение важности признаков

In [ ]:
feature_names = preprocessor.get_feature_names_out(input_features=X_train.columns)

In [ ]:
feature_importance = regressor.feature_importances_
# Сортировка признаков по важности
sorted_idx = np.argsort(feature_importance)[::-1]

# Вывод и визуализация ТОП 10 важности признаков
top_features = np.array(feature_names)[sorted_idx][:10]
plt.figure(figsize=(10, 5))
plt.title("ТОП 10 важности признаков")
plt.barh(top_features, feature_importance[sorted_idx][:10], align="center")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

### Определение важности признаков с помощью библиотеки SHAP

In [ ]:
# Инициализация объекта SHAP Explainer
explainer = shap.TreeExplainer(regressor)
# Вычисление SHAP значений для тестового набора данных
shap_values = explainer.shap_values(preprocessor.transform(X_test))
# Визуализация важности признаков
shap.summary_plot(
    shap_values,
    preprocessor.transform(X_test),
    feature_names=feature_names,
    plot_type="bar",
    max_display=10,
    plot_size=(8, 6),
    title="Важность признаков",
    show=False,
)
plt.title("Важность признаков", fontsize=12)
plt.xlabel("SHAP значение", fontsize=10)
plt.ylabel("Признаки", fontsize=10)
plt.show()

# Получение предсказания

In [ ]:
y_test_pred = best_model.predict(df_test)
len(y_test_pred)

In [ ]:
df_test["status"] = y_test_pred
df_test.head(5)

In [ ]:
resuilt_pred_2 = df_test.loc[:, ["name", "status"]]

In [ ]:
resuilt_pred_2.to_csv("./pred_output/pred_data_2.csv", index=False)